In [4]:
import os
import sys
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import math
import json
import pandas as pd


In [5]:
fname = "./ibadan_EAs_of_interest_v2/shapefile_of_Ibadan_EAs_of_interest_v2.shx"
ib_data = gpd.read_file(fname)

In [6]:
ib_data.head()

,city,ward,ea_code,EAS,Longitude,Latitude,Settlement,geometry
0,Ibadan,Agugu,Ag_001,Opposite Christ the king primary school I,3.919976,7.386031,Slum,POINT (3.91998 7.38603)
1,Ibadan,Agugu,Ag_002,Beside Christ the King Primary School I,3.920514,7.386621,Slum,POINT (3.92051 7.38662)
2,Ibadan,Agugu,Ag_003,Behind Christ the kind Primary School I,3.920634,7.385972,Slum,POINT (3.92063 7.38597)
3,Ibadan,Agugu,Ag_004,Behind The Board of Trade,3.922894,7.386947,Slum,POINT (3.92289 7.38695)
4,Ibadan,Agugu,Ag_005,Opposite The Board of Trade,3.922421,7.386521,Slum,POINT (3.92242 7.38652)


In [11]:
ib_data['ward'].unique()

array(['Agugu', 'Bashorun', 'Challenge', 'Olopomewa'], dtype=object)

In [23]:
test = ib_data.loc[(ib_data['ward'] == "Challenge") & (ib_data['Settlement'] == 'informal')]
test.shape

(105, 8)

### Specified Data

In [28]:
def columnSpecifiedBasedData(dataframe, settings):
    """
    Parameters:
        dataframe: it is a dataframe in the formate of GeoPandas of Pandas
        settings: it is a dictionary of the settings. 
    Return: 
        Dictionary: returns a dictionary
    """
    specified_area = settings['required_data']

    Settlement = {}
    if specified_area["category"] == "All":
        areas = dataframe[specified_area["column"]].unique() # get the "Settlment from the user. "

        for area in areas:
            area_result = dataframe.loc[dataframe[specified_area["column"]] == area]
            Settlement[area] = area_result
    else:
        area_result = dataframe.loc[dataframe[specified_area["column"]] == specified_area["category"]]
        Settlement[specified_area["category"]] = area_result
    return Settlement


dataframe = ib_data
settings = {
    "required_data" : {"column" : "ward",
                       "category" : "Bashorun"}
}
specified_area_geodf = columnSpecifiedBasedData(dataframe, settings)
print('Lenght of the Dictionary : ', len(specified_area_geodf))
# The data is a dictionary of GeoDataframe

Lenght of the Dictionary :  1


In [30]:
# 
column = settings["required_data"]
region_dataframe = specified_area_geodf[column["category"]]
region_dataframe.shape


(85, 8)

In [15]:
def generate_rectangle(center_lon, center_lat, width, height):
    """
    Generate the coordinates of a rectangle centered around a given location.
    
    Parameters:
    - center_lon: Longitude of the center point
    - center_lat: Latitude of the center point
    - width: Width of the rectangle in meters
    - height: Height of the rectangle in meters
    
    Returns:
    - List of coordinates representing the rectangle
    """
    # Convert meters to degrees (approximate)
    meters_to_degrees_lon = (111320 * math.cos(center_lat * math.pi / 180))
    meters_to_degrees_lat = 110574
    
    # Calculate the coordinates of the rectangle corners
    min_lon = center_lon - (width / (2 * meters_to_degrees_lon))
    max_lon = center_lon + (width / (2 * meters_to_degrees_lon))
    min_lat = center_lat - (height / (2 * meters_to_degrees_lat))
    max_lat = center_lat + (height / (2 * meters_to_degrees_lat))
    
    # Define the rectangle coordinates in counterclockwise direction
    rectangle_coordinates = [
        [min_lon, min_lat],
        [min_lon, max_lat],
        [max_lon, max_lat],
        [max_lon, min_lat],
        [min_lon, min_lat]  # Closing the loop
    ]
    
    return rectangle_coordinates

# Example usage:
# Given location

# center_lon = 3.951372252965484
# center_lat = 7.434130780421128

# # Generate rectangle with width and height of 20 meters
# rectangle_coordinates = generate_rectangle(center_lon, center_lat, 20, 20)
# print(rectangle_coordinates)

In [34]:
def df_to_json(df, settings, additional_features = False):
    """
    Convert a DataFrame with longitude and latitude columns to a JSON format with specified structure.
    
    Parameters:
    - df: DataFrame containing the data
    - setting: include all the feature that need to be included. 
    - polygon: polygon the polygon shape that needs to be created based on the given latitude and longitude
    
    Returns:
    - JSON formatted data
    """
    features = []
    coordinates = settings['extracted_features']
    properties = settings['properties_to_include']
    sparcity = settings['distance']
    additional_feature_items = settings["class_specification"]
    
    for _, row in df.iterrows():
        # Extract longitude and latitude
        lon = row[coordinates[0]]
        lat = row[coordinates[1]]
        
        # Create properties dictionary
        # Extract longitude and latitude
        lon = row[coordinates[0]]
        lat = row[coordinates[1]]
        
        # Create properties dictionary
        if properties:
            props = {prop: row[prop] for prop in properties}
            if additional_features:
                for k, v in additional_feature_items.items():
                    if k == "fill":
                        props[k] = v[(row['Settlement'])]
                    else:
                        props[k] = v
                        

        else:
            props = {}
        
        # Add longitude and latitude to properties
        props[coordinates[0]] = lon
        props[coordinates[1]] = lat
        if settings['shape_type'] == 'Polygon' or settings['shape_type'] == 'Both': 
            shape = [generate_rectangle(lon, lat, sparcity['width'], sparcity['height'])]
            # Create GeoJSON feature
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Polygon",
                    "coordinates": shape
                }
            }
            features.append(feature)
        if settings['shape_type'] == "Point" or settings['shape_type'] == 'Both': 
            shape = [lon, lat]
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Point",
                    "coordinates": shape
                }
            }


            features.append(feature)
    
    # Create FeatureCollection
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }
    
    return feature_collection

# Example usage:
# Assuming your DataFrame is named 'df' and contains columns: city, ward, ea_code, EAS, Longitude, Latitude, Settlement, geometry
# And you want to include city, ward, ea_code, EAS, Settlement in the properties
additional_features = {
        "stroke": "#232323",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": {
                "Slum" : "#FF0000",
                "slums" : "#CC0000",
                "slum" : "#FF3333",
                "Formal" : "#00FF00",
                "formal" : "#33FF33",
                "Informal" : "#0000FF",
                'informal' : "#3333FF"
        },
        "fill-opacity": 0.5
    }

settings = {
    "properties_to_include" : ["city", "ward", 'ea_code', 'EAS', 'Settlement'],
    'extracted_features' : ['Longitude', 'Latitude'],
    "distance" : {'width' : 200, 'height': 200}, # Width and height are messured in miters. 
    'shape_type' : 'Polygon', # Polygone, Point, Both
    "class_specification" : additional_features
}

json_data = df_to_json(region_dataframe, settings, additional_features = True)

# Save JSON to a file
with open('json_file.json', 'w') as f:
    json.dump(json_data, f)


In [33]:
import geojson
import pandas as pd

def df_to_geojson(df, settings, additional_features = False):
    """
    Convert a DataFrame with longitude and latitude columns to a GeoJSON format with specified structure.
    
    Parameters:
    - df: DataFrame containing the data
    - setting: include all the feature that need to be included. 
    - polygon: polygon the polygon shape that needs to be created based on the given latitude and longitude
    
    Returns:
    - GeoJSON FeatureCollection
    """
    features = []
    coordinates = settings['extracted_features']
    properties = settings['properties_to_include']
    sparcity = settings['distance']
    additional_feature_items = settings["class_specification"]
    
    for _, row in df.iterrows():
        # Extract longitude and latitude
        lon = row[coordinates[0]]
        lat = row[coordinates[1]]
        
        # Create properties dictionary
        # Extract longitude and latitude
        lon = row[coordinates[0]]
        lat = row[coordinates[1]]
        
        # Create properties dictionary
        if properties:
            props = {prop: row[prop] for prop in properties}
            if additional_features:
                for k, v in additional_feature_items.items():
                    if k == "fill":
                        
                        props[k] = v[(row['Settlement'])]
                    else:
                        props[k] = v

        else:
            props = {}
        
        # Add longitude and latitude to properties
        props[coordinates[0]] = lon
        props[coordinates[1]] = lat
        if settings['shape_type'] == 'Polygon' or settings['shape_type'] == 'Both': 
            shape = [generate_rectangle(lon, lat, sparcity['width'], sparcity['height'])]
            # Create GeoJSON feature
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Polygon",
                    "coordinates": shape
                }
            }
            features.append(feature)
        if settings['shape_type'] == "Point" or settings['shape_type'] == 'Both': 
            shape = [lon, lat]
            feature = {
                "type": "Feature",
                "properties": props,
                "geometry": {
                    "type": "Point",
                    "coordinates": shape
                }
            }


            features.append(feature)
    
    # Create FeatureCollection
    feature_collection = geojson.FeatureCollection(features)
    
    return feature_collection

# Example usage:
# Assuming your DataFrame is named 'df' and contains columns: city, ward, ea_code, EAS, Longitude, Latitude, Settlement, geometry
# And you want to include city, ward, ea_code, EAS, Settlement in the properties
additional_features = {
        "stroke": "#232323",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": {
                "Slum" : "#FF0000",
                "slums" : "#CC0000",
                "slum" : "#FF3333",
                "Formal" : "#00FF00",
                "formal" : "#33FF33",
                "Informal" : "#0000FF",
                'informal' : "#3333FF"
        },
        "fill-opacity": 0.5
    }

settings = {
    "properties_to_include" : ["city", "ward", 'ea_code', 'EAS', 'Settlement'],
    'extracted_features' : ['Longitude', 'Latitude'],
    "distance" : {'width' : 200, 'height': 200}, # Width and height are messured in miters. 
    'shape_type' : 'Polygon', # Polygone, Point, Both
    "class_specification" : additional_features
}

geojson_data = df_to_geojson(ib_data.head(), settings, additional_features = True)
# Save GeoJSON to a file
with open('geojson_file.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

    